In [1]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

In [2]:
import sys
import os

sys.path.append('tools')

import multicrop_dataset
import architecture

In [3]:
import numpy as np
import matplotlib

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

matplotlib.use("nbAgg")

import data_utility

from itertools import groupby
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
dir_list = os.listdir('data/rns_data')
patientIDs = [s for s in dir_list for type_string in ['HUP', 'RNS'] if type_string in s.upper()]

In [6]:
patientIDs[14:]

['HUP143',
 'HUP147',
 'HUP153',
 'HUP156',
 'HUP159',
 'HUP182',
 'HUP192',
 'HUP197',
 'HUP199',
 'HUP205',
 'RNS021',
 'RNS022',
 'RNS026',
 'RNS029']

In [7]:
# data_import = data_utility.read_files(path = 'data/rns_data', patientIDs=patientIDs[14:], path_data = 'rns_raw_cache',annotation_only = False, verbose=True)
# ids = list(data_import.keys())

In [8]:
# window_len = 1
# stride = 1
# concat_n = 4
# for id in tqdm(ids):
#     data_import[id].set_window_parameter(window_length=window_len, window_displacement=stride)
#     data_import[id].set_concatenation_parameter(concatenate_window_n=concat_n)
#     data_import[id].get_windowed_data(data_import[id].catalog["Event Start idx"],data_import[id].catalog["Event End idx"])
#     data_import[id].normalize_windowed_data()
#     _, concatenated_data = data_import[id].get_concatenated_data(data_import[id].windowed_data, arrange='channel_stack')
#     np.save('rns_cache/'+ id +'.npy',concatenated_data)

In [9]:
file_list = ['HUP159.npy']

In [10]:
class RNS_Raw(Dataset):
    def __init__(self, file_names, transform=True, astensor = True):
        self.file_names = file_names
        self.transform = transform
        
        file_name_temp = self.file_names[0]
        with open('rns_cache/'+ file_name_temp, 'rb') as f:
            temp_file = np.load(f)
        
        
        self.data = np.empty((0,temp_file.shape[1],temp_file.shape[2]))    
        print(self.data.shape)
        
        for name in tqdm(self.file_names):
            with open('rns_cache/'+ name, 'rb') as f:
                cache = np.load(f)
            self.data = np.vstack((self.data,cache))
        
        print
        print('data loaded')

        self.length = len(self.data)

        if astensor:
            self.augmentation = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.RandomApply([T.ColorJitter()], p=0.5),
                T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
                T.RandomInvert(p=0.2),
                T.RandomPosterize(4, p=0.2),
                T.ToTensor()
            ])

            self.totensor = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.ToTensor()
            ])
        else:
            self.augmentation = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.RandomApply([T.ColorJitter()], p=0.5),
                T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
                T.RandomInvert(p=0.2),
                T.RandomPosterize(4, p=0.2),
            ])

            self.totensor = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
            ])

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.data[index]

        if self.transform:
            concat_len = data.shape[1]/4
            channel_index = np.arange(4)
            np.random.shuffle(channel_index)
            channel_index = channel_index*concat_len+(concat_len-1)/2
            channel_index = np.repeat(channel_index, concat_len)
            concate_len_1 = (concat_len-1)/2
            a_repeat = np.arange(-concate_len_1,concate_len_1+1)[np.newaxis].T
            base_repeat = np.repeat(a_repeat, 4,axis=1).T.flatten()
            channel_index = channel_index+base_repeat
            data = data[channel_index.astype(int)]
            data = torch.from_numpy(data).clone()
            data = data.repeat(3, 1, 1)
            data = self.augmentation(data)

        else:
            data = torch.from_numpy(data).clone()
            data = data.repeat(3, 1, 1)
            data = self.totensor(data)

        return data, [], None

In [11]:
unlabeled_dataset = RNS_Raw(file_list, transform=True,astensor = False)

(0, 249, 36)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]

data loaded


In [12]:
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset, SwaVCollateFunction
from lightly.loss import SwaVLoss
from lightly.loss.memory_bank import MemoryBankModule
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes


class SwaV(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(2048, 2048, 128)
        self.prototypes = SwaVPrototypes(128, 2048,1)

        self.start_queue_at_epoch = 35
        self.queues = nn.ModuleList([MemoryBankModule(size=256) for _ in range(2)])

    def forward(self, high_resolution, low_resolution, epoch):
        self.prototypes.normalize()

        high_resolution_features = [self._subforward(x) for x in high_resolution]
        low_resolution_features = [self._subforward(x) for x in low_resolution]

        high_resolution_prototypes = [
            self.prototypes(x, epoch) for x in high_resolution_features
        ]
        low_resolution_prototypes = [
            self.prototypes(x, epoch) for x in low_resolution_features
        ]
        queue_prototypes = self._get_queue_prototypes(high_resolution_features, epoch)

        return high_resolution_prototypes, low_resolution_prototypes, queue_prototypes

    def _subforward(self, input):
        features = self.backbone(input).flatten(start_dim=1)
        features = self.projection_head(features)
        features = nn.functional.normalize(features, dim=1, p=2)
        return features

    @torch.no_grad()
    def _get_queue_prototypes(self, high_resolution_features, epoch):
        if len(high_resolution_features) != len(self.queues):
            raise ValueError(
                f"The number of queues ({len(self.queues)}) should be equal to the number of high "
                f"resolution inputs ({len(high_resolution_features)}). Set `n_queues` accordingly."
            )

        # Get the queue features
        queue_features = []
        for i in range(len(self.queues)):
            _, features = self.queues[i](high_resolution_features[i], update=True)
            # Queue features are in (num_ftrs X queue_length) shape, while the high res
            # features are in (batch_size X num_ftrs). Swap the axes for interoperability.
            features = torch.permute(features, (1, 0))
            queue_features.append(features)

        # If loss calculation with queue prototypes starts at a later epoch,
        # just queue the features and return None instead of queue prototypes.
        if self.start_queue_at_epoch > 0 and epoch < self.start_queue_at_epoch:
            return None

        # Assign prototypes
        queue_prototypes = [self.prototypes(x, epoch) for x in queue_features]
        return queue_prototypes


resnet = torchvision.models.resnet50()

backbone = nn.Sequential(*list(resnet.children())[:-1])
model = SwaV(backbone)

checkpoint = torch.load('rns_ckpt_159/checkpoint29.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.train()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

collate_fn = SwaVCollateFunction(gaussian_blur = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0, normalize={'mean':[0, 0, 0], 'std':[1, 1, 1]})

dataloader = torch.utils.data.DataLoader(
    unlabeled_dataset,
    batch_size=72,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

criterion = SwaVLoss(sinkhorn_epsilon = 0.05)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

torch.save({
            'epoch': -1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': -1,
            }, 'rns_ckpt_159/checkpoint'+str(-1)+'.pth')

print("Starting Training")
for epoch in range(30,100):
    print('epoch', epoch)
    total_loss = 0
    i = 0
    for batch, _, _ in tqdm(dataloader):
        i+=1
        batch = [x.to(device) for x in batch]
        high_resolution, low_resolution = batch[:2], batch[2:]
        high_resolution, low_resolution, queue = model(
            high_resolution, low_resolution, epoch
        )
        loss = criterion(high_resolution, low_resolution, queue)
        total_loss += loss.detach()
        if i % 50 == 1:
            avg_loss = total_loss / i
            print(f"step: {i:>02}, loss: {avg_loss:.5f}")
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(dataloader)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            }, 'rns_ckpt_159/checkpoint'+str(epoch)+'.pth')

    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

Starting Training
epoch 30


  0%|                                                                                          | 0/566 [00:00<?, ?it/s]

step: 01, loss: 5.16035


  9%|███████▏                                                                         | 50/566 [00:50<08:13,  1.05it/s]

step: 51, loss: 5.11392


...

step: 501, loss: 4.30689


 97%|█████████████████████████████████████████████████████████████████████████████▋  | 550/566 [09:09<00:15,  1.01it/s]

step: 551, loss: 4.30353


100%|████████████████████████████████████████████████████████████████████████████████| 566/566 [09:25<00:00,  1.00it/s]


epoch: 99, loss: 4.30216
